---
title: Downloading Climate Data from ERA5-Land
short_title: ERA5 Climate Data
---

In this notebook we will demonstrate how we can use Climate Tools and `dhis2eo` to retrieve climate data for a set of DHIS2 organisation units, based on the [ERA5-Land dataset](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land), hosted at the [ECMWF Climate Data Store (CDS)](https://cds.climate.copernicus.eu/datasets). 

----------------------------------------
## Requirements 

### 1. Create an ECMWF User

Before you can download the dataset programmatically, you need to [create an ECMWF user](https://www.ecmwf.int/user/login).

### 2. Authenticate with your ECMWF user

Next, you need to authenticate with this user based on your user credentials:

- Go to the [CDSAPI Setup page](https://cds.climate.copernicus.eu/how-to-api) and make sure to login.
- Once logged in, scroll down to the section "Setup the CDS API personal access token". 
  - This should show your login credentials, and look something like this:

        url: https://cds.climate.copernicus.eu/api
        key: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

- Copy those two lines to a file `.cdsapirc` in your user's $HOME directory.

### 3. Accept the dataset license

Lastly, ECMWF requires that you manually accept the user license for each dataset that you download. 

- Visit the Download page of the dataset you want to download (links below).
- Scroll down until you get to the "Terms of Use" section.
- Click the button to accept and login with your user if you haven't already. 

--------------------------------------------
## What you need

We start by importing the necessary libraries:

In [1]:
import dhis2eo
import dhis2eo.org_units
from dhis2eo.data.cds import era5_land

We also load our Sierra Leone organisation unit GeoJSON file downloaded from DHIS2. Alternative, see our [guide for how to fetch organisation units directly from your DHIS2 instance](../../org-units/download-web-api.ipynb). 

In [2]:
org_units = dhis2eo.org_units.from_file('../../data/sierra-leone-districts.geojson', dhis2=True)
org_units

,org_unit_id,name,level,geometry
0,O6uvpzGd5pu,Bo,2,"POLYGON ((-11.5914 8.4875, -11.5906 8.4769, -1..."
1,fdc6uOvgoji,Bombali,2,"POLYGON ((-11.8091 9.2032, -11.8102 9.1944, -1..."
2,lc3eMKXaEfw,Bonthe,2,"MULTIPOLYGON (((-12.5568 7.3832, -12.5574 7.38..."
3,jUb8gELQApl,Kailahun,2,"POLYGON ((-10.7972 7.5866, -10.8002 7.5878, -1..."
4,PMa2VCrupOd,Kambia,2,"MULTIPOLYGON (((-13.1349 8.8471, -13.1343 8.84..."
5,kJq2mPyFEHo,Kenema,2,"POLYGON ((-11.3596 8.5317, -11.3513 8.5234, -1..."
6,qhqAxPSTUXp,Koinadugu,2,"POLYGON ((-10.585 9.0434, -10.5877 9.0432, -10..."
7,Vth0fbpFcsO,Kono,2,"POLYGON ((-10.585 9.0434, -10.5848 9.0432, -10..."
8,jmIPBj66vD6,Moyamba,2,"MULTIPOLYGON (((-12.6351 7.6613, -12.6346 7.66..."
9,TEQlaapDQoK,Port Loko,2,"MULTIPOLYGON (((-13.119 8.4718, -13.1174 8.470..."


--------

## Downloading ERA5-Land data

The `dhis2eo.data.cds.era5_land` module allows you to easily download climate data for different time periods from the ERA5-Land dataset. 

### Hourly data

To import daily ERA5-Land data into DHIS2, we recommend downloading [ERA5-Land data at hourly levels](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land), and then aggregating to up to the daily level. This is because the [daily version of ERA5-Land](https://cds.climate.copernicus.eu/datasets/derived-era5-land-daily-statistics) does not include the precipitation variable. 

The data can be downloaded by calling the function `dhis2eo.data.cds.era5_land.get('hourly', ...)`. By default, this downloads the most commonly requested ERA5 climate variables: 2m temperature and total precipitation, but you can also set the `variables` parameters to any of the ERA5-Land variables. 

Due to the large amounts of data, this dataset only allows downloading one month at a time. We therefore need to loop over each year and month, and call the download function for each `year` and `month` value. 

For the purpose of this notebook, let's download data for the last 3 months: 

In [6]:
start_year = 2025
start_month = 7
end_year = 2025
end_month = 9

for year in range(start_year, end_year+1):
    for month in range(1, 12+1):
        
        # skip months before or after our defined time range
        if (year,month) < (start_year,start_month):
            continue
        if (year,month) > (end_year,end_month):
            continue

        # download the climate data
        # commented out for this notebook
        print(f'Month: {year}-{month}')
        hourly_data = era5_land.get('hourly', year=year, month=month, org_units=org_units)

        # do something with the data
        # e.g. save to disk, aggregate, or import to DHIS2
        # ...

Month: 2025-7
dhis2eo.data.cds.era5_land - INFO - Downloading data from CDS API...
dhis2eo.data.cds.era5_land - INFO - Request parameters: 
{"variable": ["2m_temperature", "total_precipitation"], "year": "2025", "month": ["07"], "day": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"], "time": ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"], "area": [10.0004, -13.3035, 6.9176, -10.2658], "data_format": "netcdf", "download_format": "unarchived"}


2025-11-13 19:10:03,595 INFO Request ID is 01b5c5cb-297a-4458-b873-a1a2ae6e0900
2025-11-13 19:10:03,657 INFO status has been updated to accepted
2025-11-13 19:10:17,223 INFO status has been updated to running
2025-11-13 19:14:22,324 INFO status has been updated to successful
                                                                                         

Month: 2025-8
dhis2eo.data.cds.era5_land - INFO - Downloading data from CDS API...
dhis2eo.data.cds.era5_land - INFO - Request parameters: 
{"variable": ["2m_temperature", "total_precipitation"], "year": "2025", "month": ["08"], "day": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"], "time": ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"], "area": [10.0004, -13.3035, 6.9176, -10.2658], "data_format": "netcdf", "download_format": "unarchived"}


2025-11-13 19:14:23,430 INFO Request ID is 7e418526-627b-4e59-86c0-28e54e54dcb1
2025-11-13 19:14:23,502 INFO status has been updated to accepted
2025-11-13 19:14:37,028 INFO status has been updated to running
2025-11-13 19:18:42,014 INFO status has been updated to successful
                                                                                         

Month: 2025-9
dhis2eo.data.cds.era5_land - INFO - Downloading data from CDS API...
dhis2eo.data.cds.era5_land - INFO - Request parameters: 
{"variable": ["2m_temperature", "total_precipitation"], "year": "2025", "month": ["09"], "day": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30"], "time": ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"], "area": [10.0004, -13.3035, 6.9176, -10.2658], "data_format": "netcdf", "download_format": "unarchived"}


2025-11-13 19:18:43,258 INFO Request ID is 76804933-6007-4cf8-9de7-f8e7443608bc
2025-11-13 19:18:43,343 INFO status has been updated to accepted
2025-11-13 19:18:51,767 INFO status has been updated to running
2025-11-13 19:25:02,044 INFO status has been updated to successful


To inspect the contents of the downloaded data, let's view the data of the last element in the loop: 

In [ ]:
hourly_data.to_xarray()

<xarray.Dataset> Size: 5MB
Dimensions:     (valid_time: 696, latitude: 31, longitude: 31)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 6kB 2025-09-01 ... 2025-09-29T23:...
  * latitude    (latitude) float64 248B 9.918 9.818 9.718 ... 7.118 7.018 6.917
  * longitude   (longitude) float64 248B -13.3 -13.2 -13.1 ... -10.5 -10.4 -10.3
    number      int64 8B ...
    expver      (valid_time) <U4 11kB dask.array<chunksize=(696,), meta=np.ndarray>
Data variables:
    t2m         (valid_time, latitude, longitude) float32 3MB dask.array<chunksize=(696, 31, 31), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 3MB dask.array<chunksize=(696, 31, 31), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-13T18:17 GRIB to CDM+CF via cfgrib-0.9.1...

We see that this is hourly data for the month of September 2025, and contains data variables `t2m` (2m temperature), and `tp` (total precipitation). For more information on how to aggregate from hourly to daily data, see our [guide to aggregate hourly to daily data](../../aggregation/time-period-aggregation.ipynb).  

### Monthly data

In some cases you may only be interested in monthly climate data at longer time scales. In those cases we recommend downloading the [monthly ERA5-Land data](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means). 

These data can be downloaded by calling the function `dhis2eo.data.cds.era_land.get('monthly', ...)`. Similarly to the hourly data, the default variables are temperature and precipitation, but this can be specified with the `variables` parameter. 

Since monthly data takes a lot less space, you can download the data by specifying a list of `years` and a list of `months`. 

Let's download monthly data for the last 10 years:

In [ ]:
years = list(range(2015, 2025+1))
months = list(range(1, 12+1))
monthly_data = era5_land.get('monthly', years=years, months=months, org_units=org_units)

dhis2eo.data.cds.era5_land - INFO - Downloading data from CDS API...
dhis2eo.data.cds.era5_land - INFO - Request parameters: 
{"product_type": ["monthly_averaged_reanalysis"], "variable": ["2m_temperature", "total_precipitation"], "year": ["2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024", "2025"], "month": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"], "time": ["00:00"], "area": [10.0004, -13.3035, 6.9176, -10.2658], "data_format": "netcdf", "download_format": "unarchived"}


2025-11-13 18:59:38,585 INFO Request ID is b7d585bb-97da-46ea-bb7e-9cdbfab77c2b
2025-11-13 18:59:38,653 INFO status has been updated to accepted
2025-11-13 18:59:47,040 INFO status has been updated to running
2025-11-13 19:00:54,274 INFO status has been updated to successful


And inspect the results:

In [ ]:
monthly_data.to_xarray()

<xarray.Dataset> Size: 1MB
Dimensions:     (valid_time: 130, latitude: 31, longitude: 31)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 1kB 2015-01-01 ... 2025-10-01
  * latitude    (latitude) float64 248B 9.918 9.818 9.718 ... 7.118 7.018 6.917
  * longitude   (longitude) float64 248B -13.3 -13.2 -13.1 ... -10.5 -10.4 -10.3
    number      int64 8B ...
    expver      (valid_time) <U4 2kB dask.array<chunksize=(130,), meta=np.ndarray>
Data variables:
    t2m         (valid_time, latitude, longitude) float32 500kB dask.array<chunksize=(130, 31, 31), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 500kB dask.array<chunksize=(130, 31, 31), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-13T19:00 GRIB to CDM+CF via cfgrib-0.9.1...

We see the result contains 130 months (over 10 years) of data for temperature and precipitation variables. 

## Next steps

This notebook has showed how to download the ERA5 climate data. For guidance on what to do with the data after downloading, see: 

* [Aggregate the data to your organisation units](../../aggregation/org-unit-aggregation.ipynb).
* [Import the aggregated data to DHIS2](../../import-data/using-python-client.ipynb).

As these are daily updated data, this script should be run regularly to continue feeding DHIS2 with the latest data. This can be done by automatically triggering the script to be run at regular intervals, e.g. via a `cron` job (guide to be added in the future). 